In [68]:
# Import packages

import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns

In [69]:
# Matplotlib settings & rng object

rng = np.random.default_rng(2020)

%matplotlib qt

SMALL_SIZE = 14
MEDIUM_SIZE = 18
BIGGER_SIZE = 22
CHONK_SIZE = 32
font = {'family' : 'DIN Condensed',
        'weight' : 'bold',
        'size'   : SMALL_SIZE}
plt.rc('font', **font)
plt.rc('axes', titlesize=BIGGER_SIZE, labelsize=MEDIUM_SIZE, facecolor="xkcd:light grey")
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=CHONK_SIZE, facecolor="xkcd:ice blue", edgecolor="xkcd:black") #  powder blue

In [70]:
# Import data

df = pd.read_csv('ionosphere.data', header=None)

X = df.copy().iloc[:, :-1]
X

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,33
0,1,0,0.99539,-0.05889,0.85243,0.02306,0.83398,-0.37708,1.00000,0.03760,...,0.56811,-0.51171,0.41078,-0.46168,0.21266,-0.34090,0.42267,-0.54487,0.18641,-0.45300
1,1,0,1.00000,-0.18829,0.93035,-0.36156,-0.10868,-0.93597,1.00000,-0.04549,...,-0.20332,-0.26569,-0.20468,-0.18401,-0.19040,-0.11593,-0.16626,-0.06288,-0.13738,-0.02447
2,1,0,1.00000,-0.03365,1.00000,0.00485,1.00000,-0.12062,0.88965,0.01198,...,0.57528,-0.40220,0.58984,-0.22145,0.43100,-0.17365,0.60436,-0.24180,0.56045,-0.38238
3,1,0,1.00000,-0.45161,1.00000,1.00000,0.71216,-1.00000,0.00000,0.00000,...,1.00000,0.90695,0.51613,1.00000,1.00000,-0.20099,0.25682,1.00000,-0.32382,1.00000
4,1,0,1.00000,-0.02401,0.94140,0.06531,0.92106,-0.23255,0.77152,-0.16399,...,0.03286,-0.65158,0.13290,-0.53206,0.02431,-0.62197,-0.05707,-0.59573,-0.04608,-0.65697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346,1,0,0.83508,0.08298,0.73739,-0.14706,0.84349,-0.05567,0.90441,-0.04622,...,0.95378,-0.04202,0.83479,0.00123,1.00000,0.12815,0.86660,-0.10714,0.90546,-0.04307
347,1,0,0.95113,0.00419,0.95183,-0.02723,0.93438,-0.01920,0.94590,0.01606,...,0.94520,0.01361,0.93522,0.04925,0.93159,0.08168,0.94066,-0.00035,0.91483,0.04712
348,1,0,0.94701,-0.00034,0.93207,-0.03227,0.95177,-0.03431,0.95584,0.02446,...,0.93988,0.03193,0.92489,0.02542,0.92120,0.02242,0.92459,0.00442,0.92697,-0.00577
349,1,0,0.90608,-0.01657,0.98122,-0.01989,0.95691,-0.03646,0.85746,0.00110,...,0.91050,-0.02099,0.89147,-0.07760,0.82983,-0.17238,0.96022,-0.03757,0.87403,-0.16243


### 3.1: Perform PCA on the data and make a scatter plot of PC1 and PC2 (the first two principal compo- nents). Are PC1 and PC2 linearly correlated?

No, after viewing corrplot for the data we can see they are orthogonal (as expected), with correlation between PCs at literally 0 if not for damnable floating point errors.

In [71]:
# Q3.1

plt.clf()

pca_2 = PCA(n_components=2)
pca_2.fit(X)

X_new_2 = pca_2.transform(X)
X_new_2

df_pca_2 = pd.DataFrame(X_new_2)
df_pca_2.columns = ['PC1', 'PC2']
corr = round(df_pca_2.corr(), 5)
fig = sns.heatmap(corr, annot=True)
plt.title("Correlation Matrix for PCs")
fig.set_yticklabels(fig.get_yticklabels(), rotation = 0)
plt.show()

### 3.2: There are three methods to pick a set of principle components: (1) In the plot where the curve bends (elbow technique); (2) Add the percentage variance until total 75% is reached (70 − 90%) (3) Use the components whose variance is at least one. Report the components selected for each technique.

1. Elbow Technique
    * I selected the first 7 PCs using this method, though you could make an argument for more or less than mine. To justify my selection I will include the plot of both actual component variance and cumulative component variance for all PCs.
<br><br>
2. Add variance until total
    * With a threshold of 75%, we need to use the first 10 PCs.
<br><br>
3. Use components with variance >= 1
    * Only the first two PCs have variance greater than 1, so we only use those two.

In [72]:
# Setup for Q3.2

pca_all = PCA(n_components=X.shape[1])
pca_all.fit(X)
component_ratios = pca_all.explained_variance_ratio_
components = pca_all.explained_variance_

x = range(1, len(component_ratios) + 1)
vars = component_ratios
cum_vars = component_ratios.cumsum()

In [73]:
# Q3.2.1
# plt.clf()

fig, (ax1, ax2) = plt.subplots(ncols=2, sharex=True)
sns.lineplot(x = x, y = vars*100, marker="o", markerfacecolor="red",  ax=ax1)
sns.lineplot(x = x, y = cum_vars*100, marker="o", markerfacecolor="red", ax=ax2)
ax1.set_title('Component Variance')
ax2.set_title('Cumulative Variance')
ax1.set_xticks(range(1, X.shape[1]+1, 2))
ax1.set_xlabel("PC Number")
ax2.set_xlabel("PC Number")
ax1.set_ylabel("% Variance")
ax2.set_ylabel("Cumulative % Variance")
# ax2.set_xticks(range(1, X.shape[1]+1, 2))

plt.show()

In [74]:
# Q3.2.2
def find_components_for_threshold(threshold):
    total = 0
    inc = 1
    while total < threshold:
        total += component_ratios[inc - 1]
        inc += 1

    return inc

print(f"Number of components needed to explain 75% of the variance: {find_components_for_threshold(0.75)}")

Number of components needed to explain 75% of the variance: 10


In [75]:
# Q3.2.3

[round(x, 5) for x in components]

component_list = []
for i, component in enumerate(components):
    if component >= 1:
        component_list.append(component)

print(f"Number of components included with variance greater than 1: {len(component_list)}")

Number of components included with variance greater than 1: 2


### 3.3: Report and discuss loadings in PCA such as, using prcomp() or princomp() in R. How are principal components and original variables related?

I am using Python instead of R, so I will discuss the PCA object from sklearn.decomposition. This object has a few methods we will find most useful. First, to simply fit pca to the data, use the PCA.fit() method with n_components set as the number of components to keep after PCA. This returns the parameters we are looking for in the previous question; we can use it to find the variance explained by each component, the ratio of that variance, and a few other useful pieces of information. 

To actually transform our data, however, we will use a method of PCA that uses the fitted model (PCA.fit()) to apply a linear transformation to the data according to the calculated coefficients. After both fitting and transforming the data, we have n principal components remaining.

One important difference between the PCA method we learned and the PCA method from sklearn is that, as most other implementations that can be found, the sklearn uses Singular Value Decomposition (SVD) rather than Eigen Decomposition (which we learned in class). Similarly, the difference between prcomp(SVD) and princomp(Eigen) is just the method of solving. These normally yield almost identical results, but it seems that SVD is standard across implementations I have found. 

### 3.4: Perform PCA over Ionosphere data set. Keep 90% of variance after PCA and reduce Ionosphere data set and call this data ∆R. Cluster ∆R using your k-means program from previous assignment and report the total error rates for k = 2, . . . , 5 for 20 runs each. Plots are generally a good way to convey complex ideas quickly, i.e., box plots, whisker plots. Discuss your results, i.e., Does clustering get better after PCA?

According to our results here, PCA reduces SSE linearly. In this case, it reduced SSE for some k by approximately 300.

In [76]:
# Q3.4: K-Means

# plt.clf()

PCs_for_90pct_var = find_components_for_threshold(0.90)

pca_90pct = PCA(n_components=PCs_for_90pct_var)
delta_R = pd.DataFrame(pca_90pct.fit_transform(X))

def get_SSE(data, k):
    run_outputs = []
    for run in range(20):
        kmeans = KMeans(n_clusters=k).fit(data)
        run_outputs.append(kmeans.inertia_)

    return np.mean(run_outputs)

X_SSE_lst = [get_SSE(X, k) for k in range(2, 6)]
delta_R_SSE_lst = [get_SSE(delta_R, k) for k in range(2, 6)]

fig, ax = plt.subplots()
sns.lineplot(x = range(2, 6), y = X_SSE_lst, marker = "o", markerfacecolor = "red", label = "Original", ax = ax)
sns.lineplot(x = range(2, 6), y = delta_R_SSE_lst, marker = "o", markerfacecolor = "red", label = "PCA Transformed", ax = ax)
ax.set_xticks(range(2, 6))
ax.set_title("Error as a Function of K for Original and Transformed Data")
ax.set_xlabel("K Clusters")
ax.set_ylabel("Average SSE (20 trials)")

plt.show()


### Extra Credit

#### Chapter 8 Exercise 18

* Wards: As any description of HAC (of which Ward's Linkage is one subset) will tell you, this method of clustering is greedy and so will return locally optimal clusters.
* Bisecting KMeans: Similar to KMeans below, this method simply merges Hierachical Divisive Clustering with KMeans, so it retains the element of randomness in centroid initialization at every split made in the algorithm, causing it to produce locally optimal solutions.
* Standard KMeans: Because this involves randomized initial centroids, there is an element of unpredictability in the final clusters and it is only locally optimized most of the time.

#### Chapter 8 Exercise 30

* Interpreting the results of KMeans (or any clustering) requires an understanding of the input data; if we are feeding in only a matrix of terms (above a certain threshold of frequency) and how often they were used, any clustering on the data will simply group terms that occur similar amounts. Think about it; with only 1 dimension to go on (how often a term occurs), we are basically just placing terms on a number line and drawing k circles around the terms that are close together. As long as the proximity metric used stays the same, the only difference between this and determing clusters based on most popular clusters would be the number of clusters formed. 

* You could define each cluster as a class and group documents by which class most of their terms fall into, so essentially performing clustering for individual terms and documents separately.